In [2]:
using JuMP
using Gurobi
model = Model(Gurobi.Optimizer)

Set parameter Username
Set parameter LicenseID to value 2595637
Academic license - for non-commercial use only - expires 2025-12-05


A JuMP Model
├ solver: Gurobi
├ objective_sense: FEASIBILITY_SENSE
├ num_variables: 0
├ num_constraints: 0
└ Names registered in the model: none

In [7]:
# Import the necessary libraries
using HTTP
using JSON

# Store the URL
url = "https://fireroad.mit.edu/requirements/get_json/major6-4/"

# Send the request and get the response
reqs = HTTP.get(url)

# Parse the JSON response
datafile = JSON.parse(String(reqs.body))

Dict{String, Any} with 7 entries:
  "short-title"     => "6-4"
  "medium-title"    => "6-4 Major"
  "title-no-degree" => "Artificial Intelligence and Decision Making"
  "reqs"            => Any[Dict{String, Any}("connection-type"=>"all", "thresho…
  "list-id"         => "major6-4.reql"
  "title"           => "Bachelor of Science in Artificial Intelligence and Deci…
  "desc"            => "The Bachelor of Science in Artificial Intelligence and …

In [8]:
# Create a mapping for comparison types
type_mapping = Dict("LT" => "lt", "GT" => "gt", "GTE" => "geq", "LTE" => "leq")

function create_requirements(data, connector=false, cutoff=1, ineq="geq")
    mandatory = []
    all_reqs = []
    together = false
    comb_req = []
    
    for section in data["reqs"]
        ineq = "geq"
        cutoff = 1
        
        # Base case: if there's a "req" key in the section
        if haskey(section, "req")
            if connector
                push!(mandatory, section["req"])
                continue
            else
                push!(all_reqs, section["req"])
                continue
            end
        end

        # Recursive case: checking for special thresholds
        if haskey(section, "distinct-threshold")
            cutoff = section["distinct-threshold"]["cutoff"]
            ineq = type_mapping[section["distinct-threshold"]["type"]]
            if haskey(section, "threshold") && section["threshold"]["cutoff"] != cutoff
                together = true
                total_cutoff = section["threshold"]["cutoff"]
                total_ineq = type_mapping[section["threshold"]["type"]]
            end
        elseif haskey(section, "threshold")
            cutoff = section["threshold"]["cutoff"]
            ineq = type_mapping[section["threshold"]["type"]]
        end
        
        # Building based on connection types
        if section["connection-type"] == "all"
            next = create_requirements(section, true)
            if together
                comb_req = vcat(comb_req, reduce(vcat, getindex.(next, 1)))
            else
                append!(all_reqs, next)
            end
        elseif section["connection-type"] == "any"
            next = create_requirements(section)
            if together
                comb_req = vcat(comb_req, reduce(vcat, getindex.(next, 1)))
            else
                one_req = (next, ineq, cutoff)
                push!(all_reqs, one_req)
            end
        else
            return create_requirements(section)
        end
        
        if together
            push!(all_reqs, (comb_req, total_ineq, total_cutoff))
        end
    end

    if !isempty(mandatory)
        push!(all_reqs, (mandatory, "eq", length(mandatory)))
    end
    
    return all_reqs
end

# Call the function
requirements = create_requirements(datafile)

10-element Vector{Any}:
 (Any["6.100A", "6.100L"], "geq", 1)
 (Any["6.S084", "18.C06", "18.06"], "geq", 1)
 (Any["6.3700", "6.3800", "18.05"], "geq", 1)
 (Any["6.1200", "6.1010", "6.1210"], "eq", 3)
 (Any[(Any["6.3720", "6.3900", (Any["6.C01", "6.S052"], "eq", 2), (Any["6.C51", "6.S952"], "eq", 2), "6.S059"], "geq", 1), (Any["6.3000", "6.4110", "6.4400"], "geq", 1), (Any["6.3100", "6.4110", "6.7201"], "geq", 1), (Any["6.1220", "6.4400", "6.7201"], "geq", 1), (Any["6.3260", "6.3950", "6.4120", "6.4590", "6.C35", "6.S041", "9.660"], "geq", 1)], "geq", 5)
 (Any["6.1800", "6.2040", "6.2060", "6.2061", "6.2220", "6.2221", "6.2370", "6.2600", "6.4200", "6.4210", "6.4590", "6.4860", "6.4880", "6.8301", "6.8611", "6.9030", "6.UAR", "6.UAT"], "geq", 2)
 (Any["6.3900", "6.3950", "6.4590", "6.8301", "6.8611"], "geq", 1)
 (Any["6.4200", "6.4210", "6.8301", "6.8611"], "geq", 1)
 (Any["18.404", "6.3730", "6.4210", "6.5151", "6.5831", "6.7411", "6.7930", "6.8300", "6.8301", "6.8371", "6.8611", "6.870

In [ ]:
# Helper function to map the inequality string to a JuMP constraint
function create_inequality_constraint(model, expr, ineq::String, value)
    if ineq == "geq"
        @constraint(model, expr >= value)
    elseif ineq == "eq"
        @constraint(model, expr == value)
    elseif ineq == "leq"
        @constraint(model, expr <= value)
    else
        error("Unknown inequality type: $ineq")
    end
end

# Main function to create JuMP constraints based on the requirements vector
function add_requirements_constraints(model::Model, requirements::Vector, course_vars::Dict{String, JuMP.VariableRef})
    for req in requirements
        courses, ineq, cutoff = req

        # If the course requirement is a nested structure (i.e., a vector of other requirements)
        if isa(courses, Vector{Any}) && isa(courses[1], Tuple)
            # Recursively handle the nested requirements
            for nested_req in courses
                add_requirements_constraints(model, [nested_req], course_vars)
            end
        else
            # Retrieve the variables for each course in the list
            course_vars_expr = sum(get(course_vars, course, 0) for course in courses)

            # Create the appropriate inequality constraint
            create_inequality_constraint(model, course_vars_expr, ineq, cutoff)
        end
    end
end